# === Kapampangan-to-English NLLB-200 Training Pipeline (with <kap> tag) ===

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import torch
import evaluate
import numpy as np

# === 1. Config ===

In [2]:
CSV_PATH = "data/kapampangan_english.csv"
MODEL_NAME = "facebook/nllb-200-distilled-600M"
MODEL_DIR = "./kapampangan_mt_nllb"

SPECIAL_SRC_TOKEN = "<kap>"   # Custom Kapampangan language marker
TGT_LANG = "eng_Latn"         # English (Latin script)

# === 2. Load CSV ===

In [3]:
df = pd.read_csv(CSV_PATH)
df = df.rename(columns={"kapampangan": "src_text", "english": "tgt_text"})
df = df.dropna(subset=["src_text", "tgt_text"])

# === 3. Convert to HF Dataset ===

In [4]:
dataset = Dataset.from_pandas(df[["src_text", "tgt_text"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# === 4. Load Tokenizer & Model ===

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  # No src_lang yet
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Add <kap> as a special token
tokenizer.add_special_tokens({'additional_special_tokens': [SPECIAL_SRC_TOKEN]})
model.resize_token_embeddings(len(tokenizer))

# Force English output
model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(TGT_LANG)


# === 5. Preprocess ===

In [6]:
def preprocess(examples):
    # Prepend <kap> to source text
    src_texts = [f"{SPECIAL_SRC_TOKEN} {text}" for text in examples["src_text"]]

    # Tokenize source
    model_inputs = tokenizer(
        src_texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Tokenize target
    labels = tokenizer(
        examples["tgt_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )["input_ids"]

    # Replace pad token IDs with -100
    labels = [[(t if t != tokenizer.pad_token_id else -100) for t in label] for label in labels]
    model_inputs["labels"] = labels

    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

# === 6. Training Args ===

In [7]:
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    # Replace -100 with pad_token_id so metric can handle it
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    accuracy = metric_acc.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy["accuracy"]}

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_DIR,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=15,
    weight_decay=0.01,
    predict_with_generate=True,
    save_total_limit=2,
    logging_dir="./logs",
    eval_steps=500,       # evaluate every 500 steps
    logging_steps=500,    # log every 500 steps
    save_steps=500,       # save every 500 steps
    fp16=torch.cuda.is_available(),
)

# === 7. Trainer ===

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


C:\Users\Angelica\AppData\Local\Temp\ipykernel_9580\2430946255.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# === 8. Train ===

In [9]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.179400
1000,0.254100
1500,0.072600
2000,0.027400
2500,0.012300
3000,0.007000


c:\Users\Angelica\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'forced_bos_token_id': 256047}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3480, training_loss=0.22363907773604338, metrics={'train_runtime': 33227.4144, 'train_samples_per_second': 0.418, 'train_steps_per_second': 0.105, 'total_flos': 3762635365416960.0, 'train_loss': 0.22363907773604338, 'epoch': 15.0})

# === 9. Save ===

In [10]:
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
print(f"✅ Model saved to: {MODEL_DIR}")

✅ Model saved to: ./kapampangan_mt_nllb


# === 10. Translation Function ===

In [11]:
def batch_translate(texts, batch_size=8):
    results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for i in range(0, len(texts), batch_size):
        src_texts = [f"{SPECIAL_SRC_TOKEN} {t}" for t in texts[i:i+batch_size]]
        inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        results.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    return results

# === 11. Evaluate BLEU ===

In [12]:
print("\n--- Evaluating BLEU Score ---")
bleu = evaluate.load("bleu")

test_df = dataset["test"].to_pandas()
preds = batch_translate(test_df["src_text"].tolist())
refs = [[x] for x in test_df["tgt_text"].tolist()]

bleu_score = bleu.compute(predictions=preds, references=refs)
print(" BLEU Score:", bleu_score)


--- Evaluating BLEU Score ---
 BLEU Score: {'bleu': 0.29751167185801497, 'precisions': [0.5785907859078591, 0.32395498392282956, 0.23122529644268774, 0.18076923076923077], 'brevity_penalty': 1.0, 'length_ratio': 1.021453287197232, 'translation_length': 1476, 'reference_length': 1445}


# === 12. Manual Test ===

In [13]:
print("\n--- Manual Test ---")
sample_texts = [
    "Ali ku balu",
    "Anya ka?",
    "Masanting ya ing panaun ngeni",
    "E ku makanyan",
]

for i, kap_text in enumerate(sample_texts):
    translated = batch_translate([kap_text])[0]
    print(f"[{i+1}] Kapampangan: {kap_text}")
    print(f"    ➤ English: {translated}")


--- Manual Test ---
[1] Kapampangan: Ali ku balu
    ➤ English: I don't know.
[2] Kapampangan: Anya ka?
    ➤ English: Are you hungry?
[3] Kapampangan: Masanting ya ing panaun ngeni
    ➤ English: It's a good-looking day now.
[4] Kapampangan: E ku makanyan
    ➤ English: I'm not that old.
